<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bispectrum-counter-example" data-toc-modified-id="Bispectrum-counter-example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bispectrum counter example</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#optimize-structures" data-toc-modified-id="optimize-structures-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>optimize structures</a></span></li></ul></li><li><span><a href="#feature-computation" data-toc-modified-id="feature-computation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>feature computation</a></span><ul class="toc-item"><li><span><a href="#$\rho_{i}^{\otimes-3}$" data-toc-modified-id="$\rho_{i}^{\otimes-3}$-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>$\rho_{i}^{\otimes 3}$</a></span></li><li><span><a href="#$\rho_{i-i_1-i_2}^{\otimes-1}$" data-toc-modified-id="$\rho_{i-i_1-i_2}^{\otimes-1}$-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>$\rho_{i i_1 i_2}^{\otimes 1}$</a></span></li><li><span><a href="#check-degeneracy-of-$\rho_{i}^{\otimes-3}$" data-toc-modified-id="check-degeneracy-of-$\rho_{i}^{\otimes-3}$-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>check degeneracy of $\rho_{i}^{\otimes 3}$</a></span></li></ul></li><li><span><a href="#set-up-an-encoder-decoder-like-architecture" data-toc-modified-id="set-up-an-encoder-decoder-like-architecture-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>set up an encoder-decoder like architecture</a></span></li></ul></li></ul></div>

# Bispectrum counter example

In [1]:
%load_ext autoreload
%autoreload 2 

In [7]:
from ase.io import read,write
from ase.visualize import view
import numpy as np 

from equistore import Labels, TensorBlock, TensorMap
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal

import matplotlib.pyplot as plt

from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
import ase 
import tqdm 

In [8]:
from utils.mp_utils import *

In [9]:
from generate_bispectrum_structures import generate_nu3_degen_structs, analytical_partial_derivative

In [11]:
import pyscf
from pyscf import gto
from pyscf import scf,dft

def pyscf_rks_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
    m = scf.RKS(mol)
#     m = scf.RHF(mol)
    m.xc="pbe"
    m.max_cycle=600
    m.conv_tol=1e-9
    m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

def pyscf_rhf_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
#     m = scf.RKS(mol)
    m = scf.RHF(mol)
#     m.xc="pbe"
    m.max_cycle=500
    m.conv_tol=1e-9
#     m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

/scratch/nigam/anaconda3/envs/py3.7/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [14]:
# Advice on choosing the basis sets for different calculations 

#https://sites.google.com/site/orcainputlibrary/basis-sets?pli=1

In [29]:
np.random.seed(12345)
ngen = 2000
frames = []
for f in tqdm.tqdm(range(ngen)):
#     print(i)
    for i in range(1000):
        r = np.random.uniform(0.5,2)
        z1 = np.random.uniform(0.5,2)
        z2 = np.random.uniform(1.5,1.8)
        psi = np.random.uniform(0,np.pi)
        phi1 = np.random.uniform(np.pi/6, np.pi)
        phi2 = np.random.uniform(np.pi/6, np.pi)
        fr = generate_nu3_degen_structs(r, [0, phi1, phi1+phi2], 
                                   psi, z1, z2, "B", "B", "B")
        dist = fr[0].get_all_distances()
        if dist[0,1:].min()>1.5 and dist[0,1:].max()<1.8 and np.abs(dist-1.0).flatten().min()>0.5:
            fr[0].info["pars"] = str((f, "+")+(r, z1, z2, psi, phi1, phi2))
            fr[1].info["pars"] = str((f, "-")+(r, z1, z2, psi, phi1, phi2))
#             print(i)
            break
    if (i>900):
        print("oops!")
    frames += fr

100%|███████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:24<00:00, 81.02it/s]


In [31]:
en = []
idx_c=[]
for idx, f in enumerate(frames[:]): 
    try:
        jj = pyscf_rhf_calc(f)
        en.append(jj)
        print(idx, "done \n")
    except Exception as e: 
        print(e, idx, "\n")
        idx_c.append(idx)
        continue

converged SCF energy = -196.836684300522
0 done 

converged SCF energy = -196.834295077279
1 done 

converged SCF energy = -196.803667145577
2 done 

converged SCF energy = -196.803682038554
3 done 

converged SCF energy = -196.65631351809
4 done 

converged SCF energy = -196.66027657988
5 done 

converged SCF energy = -196.801348997836
6 done 

converged SCF energy = -196.792571382036
7 done 

converged SCF energy = -196.786024655078
8 done 

converged SCF energy = -196.786027277085
9 done 

converged SCF energy = -196.687750882951
10 done 

converged SCF energy = -196.698587361076
11 done 

converged SCF energy = -196.761817203976
12 done 

converged SCF energy = -196.761756271537
13 done 

SCF not converged.
SCF energy = -196.685542615006
14 done 

converged SCF energy = -196.696740239812
15 done 

converged SCF energy = -196.750935989818
16 done 

converged SCF energy = -196.71173338116
17 done 

converged SCF energy = -196.733824042088
18 done 

converged SCF energy = -196.7338565

converged SCF energy = -196.67152041016
159 done 

converged SCF energy = -196.742343640546
160 done 

converged SCF energy = -196.747624541108
161 done 

converged SCF energy = -196.772327436738
162 done 

converged SCF energy = -196.752272045948
163 done 

converged SCF energy = -196.736812512675
164 done 

converged SCF energy = -196.736546591175
165 done 

converged SCF energy = -196.787901381769
166 done 

converged SCF energy = -196.782127384215
167 done 

converged SCF energy = -196.778470095017
168 done 

converged SCF energy = -196.787640577243
169 done 

converged SCF energy = -196.81448554198
170 done 

converged SCF energy = -196.809849979059
171 done 

converged SCF energy = -196.747749372367
172 done 

converged SCF energy = -196.766504639148
173 done 

converged SCF energy = -196.758238670887
174 done 

converged SCF energy = -196.763832379637
175 done 

converged SCF energy = -196.739593538401
176 done 

converged SCF energy = -196.738376699943
177 done 

converged SCF 

316 done 

converged SCF energy = -196.745615966979
317 done 

converged SCF energy = -196.679959609249
318 done 

converged SCF energy = -196.662169228065
319 done 

converged SCF energy = -196.749906066009
320 done 

converged SCF energy = -196.746429812183
321 done 

converged SCF energy = -196.797270707873
322 done 

converged SCF energy = -196.797030191638
323 done 

converged SCF energy = -196.843958071253
324 done 

converged SCF energy = -196.827601021667
325 done 

converged SCF energy = -196.74414952646
326 done 

converged SCF energy = -196.766474555008
327 done 

converged SCF energy = -196.799046290193
328 done 

converged SCF energy = -196.801892812469
329 done 

converged SCF energy = -196.714870852754
330 done 

converged SCF energy = -196.715119447468
331 done 

converged SCF energy = -196.761576294496
332 done 

converged SCF energy = -196.750657093565
333 done 

converged SCF energy = -196.686029021659
334 done 

converged SCF energy = -196.698422440209
335 done 

co

474 done 

converged SCF energy = -167.465577232465
475 done 

converged SCF energy = -196.693766702354
476 done 

converged SCF energy = -196.69385973122
477 done 

converged SCF energy = -196.743044776582
478 done 

converged SCF energy = -196.713896764445
479 done 

converged SCF energy = -196.798449899893
480 done 

converged SCF energy = -196.801355430575
481 done 

converged SCF energy = -196.739772804412
482 done 

converged SCF energy = -196.738196618992
483 done 

converged SCF energy = -196.723296028025
484 done 

converged SCF energy = -196.74961178814
485 done 

converged SCF energy = -196.799812649088
486 done 

converged SCF energy = -196.801880788155
487 done 

converged SCF energy = -196.713215119833
488 done 

converged SCF energy = -196.691918820847
489 done 

converged SCF energy = -196.759587818178
490 done 

converged SCF energy = -196.735174867361
491 done 

converged SCF energy = -196.717759074794
492 done 

converged SCF energy = -196.735427527729
493 done 

con

632 done 

converged SCF energy = -196.690534852811
633 done 

converged SCF energy = -196.865778162115
634 done 

converged SCF energy = -196.8576972147
635 done 

converged SCF energy = -196.677817690697
636 done 

converged SCF energy = -196.656944557694
637 done 

converged SCF energy = -196.777637436802
638 done 

converged SCF energy = -196.777709493577
639 done 

converged SCF energy = -196.690273783653
640 done 

converged SCF energy = -196.692230476145
641 done 

converged SCF energy = -196.699310923994
642 done 

converged SCF energy = -196.681088409244
643 done 

converged SCF energy = -196.777789669715
644 done 

converged SCF energy = -196.785284514386
645 done 

converged SCF energy = -196.784179432092
646 done 

converged SCF energy = -196.788213226001
647 done 

converged SCF energy = -196.826625784828
648 done 

converged SCF energy = -196.831904787628
649 done 

converged SCF energy = -196.718624990436
650 done 

converged SCF energy = -196.719952482682
651 done 

con

...

3272 done 

converged SCF energy = -196.746335904776
3273 done 

converged SCF energy = -196.713890556066
3274 done 

converged SCF energy = -196.740613711222
3275 done 

converged SCF energy = -196.803351119482
3276 done 

converged SCF energy = -196.812370963729
3277 done 

converged SCF energy = -196.750943612359
3278 done 

converged SCF energy = -196.710255541445
3279 done 

converged SCF energy = -196.772550191355
3280 done 

converged SCF energy = -196.808307860961
3281 done 

converged SCF energy = -196.73489750945
3282 done 

converged SCF energy = -196.715372789879
3283 done 

converged SCF energy = -196.733123893253
3284 done 

converged SCF energy = -196.711525345395
3285 done 

converged SCF energy = -196.794848641096
3286 done 

converged SCF energy = -196.794301677995
3287 done 

converged SCF energy = -196.758016455912
3288 done 

converged SCF energy = -196.77214805714
3289 done 

converged SCF energy = -196.80558486689
3290 done 

converged SCF energy = -196.801928604

3426 done 

converged SCF energy = -196.770354840452
3427 done 

converged SCF energy = -196.797480210628
3428 done 

converged SCF energy = -196.789320004881
3429 done 

converged SCF energy = -196.668683007008
3430 done 

converged SCF energy = -196.668506499305
3431 done 

converged SCF energy = -196.782180016548
3432 done 

converged SCF energy = -196.787726905403
3433 done 

converged SCF energy = -196.86210237271
3434 done 

converged SCF energy = -196.855394677844
3435 done 

converged SCF energy = -132.394435426652
3436 done 

converged SCF energy = -132.392767837151
3437 done 

converged SCF energy = -196.831276790238
3438 done 

converged SCF energy = -196.829138010326
3439 done 

converged SCF energy = -196.683013609122
3440 done 

converged SCF energy = -196.68339790792
3441 done 

converged SCF energy = -196.715861769712
3442 done 

converged SCF energy = -196.722676014034
3443 done 

converged SCF energy = -196.777865849025
3444 done 

converged SCF energy = -196.77630221

3581 done 

converged SCF energy = -196.830937427704
3582 done 

converged SCF energy = -196.83003819238
3583 done 

converged SCF energy = -196.805210445417
3584 done 

converged SCF energy = -196.806805940519
3585 done 

converged SCF energy = -196.799233836999
3586 done 

converged SCF energy = -196.804123152459
3587 done 

converged SCF energy = -196.802420510027
3588 done 

converged SCF energy = -196.803507293239
3589 done 

converged SCF energy = -196.698646789004
3590 done 

converged SCF energy = -196.6990857598
3591 done 

converged SCF energy = -196.718315730721
3592 done 

converged SCF energy = -196.713339508522
3593 done 

converged SCF energy = -175.701330728123
3594 done 

converged SCF energy = -175.702691145145
3595 done 

converged SCF energy = -196.75779982424
3596 done 

converged SCF energy = -196.76648799465
3597 done 

converged SCF energy = -182.901154610678
3598 done 

converged SCF energy = -182.901399932177
3599 done 

converged SCF energy = -196.79874739356

3736 done 

converged SCF energy = -196.729504029261
3737 done 

converged SCF energy = -196.789323948916
3738 done 

converged SCF energy = -196.789335035397
3739 done 

converged SCF energy = -196.755318866287
3740 done 

converged SCF energy = -196.757160980059
3741 done 

converged SCF energy = -196.758523675854
3742 done 

converged SCF energy = -196.774330195065
3743 done 

converged SCF energy = -196.644665448165
3744 done 

converged SCF energy = -196.642081234942
3745 done 

converged SCF energy = -196.690854290462
3746 done 

converged SCF energy = -196.698848218688
3747 done 

converged SCF energy = -196.758103023646
3748 done 

converged SCF energy = -196.756121519855
3749 done 

converged SCF energy = -196.745903076167
3750 done 

converged SCF energy = -196.723862427123
3751 done 

converged SCF energy = -196.685836425439
3752 done 

converged SCF energy = -196.66419798241
3753 done 

SCF not converged.
SCF energy = -196.760674443213
3754 done 

converged SCF energy = -19

converged SCF energy = -196.72927747916
3891 done 

converged SCF energy = -196.795963675418
3892 done 

converged SCF energy = -196.796834004122
3893 done 

converged SCF energy = -196.729526335438
3894 done 

converged SCF energy = -196.740168933798
3895 done 

SCF not converged.
SCF energy = -196.7621841495
3896 done 

converged SCF energy = -196.746297938015
3897 done 

converged SCF energy = -196.714480531217
3898 done 

converged SCF energy = -196.714265884464
3899 done 

converged SCF energy = -196.829905249048
3900 done 

converged SCF energy = -196.832464620729
3901 done 

converged SCF energy = -196.719394860224
3902 done 

converged SCF energy = -196.725264270983
3903 done 

converged SCF energy = -196.749355051072
3904 done 

converged SCF energy = -196.75825125416
3905 done 

converged SCF energy = -196.773778257407
3906 done 

converged SCF energy = -196.763806918446
3907 done 

converged SCF energy = -196.779830279179
3908 done 

converged SCF energy = -196.7228261103
39

In [34]:
# idx_pop =[14, 15, 44, 45, 54, 55, 58, 59, 140, 141, 142, 143, 226, 227, 298, 299, 350, 351, 418, 419 ,590, 591,
#           824, 825, 858, 859, 972, 973, 1054, 1055,1294, 1295,1340, 1341, 1350, 1351, 1428, 1429,1464,1465, 
#           1546, 1547,1624, 1625, 1682, 1683, 1800, 1801, 1824, 1825, 1830, 1831, 1850, 1851, 1890, 1891, 
#           1900, 1901, 1942, 1943, 2088, 2089, 2120, 2121, 2374, 2375, 2458, 2459, 2508, 2509, 2540, 2541, 
#           2570, 2571, 2730, 2731, 2732, 2733, 2924, 2925, 3026, 3027, 3030, 3031, 3062,3063, 3068, 3069, 
#           3120, 3121,3142,3143, 3268, 3269, 3304, 3305, 3326,3327, 3342,3343, 3376,3377, 3406, 3407, 3510, 3511, 
#           3616, 3617, 3656, 3657, 3676, 3677, 3754, 3755, 3756, 3757, 3846, 3847, 3858, 3859, 3874, 3875, 3896, 
#           3897, 3954, 3955]

# idx_pop.reverse()
# for i in idx_pop: 
#     frames.pop(i)
#     en.pop(i)

In [53]:
for i,f in enumerate(frames):
#     print(frames[i].info["energy_hf"])
    f.info["energy_hf"] = en[i]

In [51]:
from ase.io import write
write("/home/nigam/scratch/MAC/bispectrum_counter/boron8_3874.xyz", frames)


### optimize structures 

In [55]:
def get_params(frame):
    parameters = []
    for x in frame.info["pars"].split()[2:]:
        parameters.append(float(x[:-1]))
    return parameters

In [56]:
delta_r = np.linspace(-0.5, 0.5, 10)

f = frames[0].copy()
for dr in delta_r: 
    r, z1, z2, psi, phi1, phi2 = get_params(f)
    par = [r+dr, [0,phi1, phi2], psi, z1, z2, 'B','B','B' ]
    fr = generate_nu3_degen_structs(r+dr, [0, phi1, phi1+phi2], 
                                   psi, z1, z2, "B", "B", "B")
    en_plus = pyscf_rhf_calc(fr[0])
    en_minus = pyscf_rhf_calc(fr[1])
    delta_e = abs(en_plus - en_minus)
    print(delta_e)
    der = analytical_partial_derivative(par, diff_param= 'r' )[0]
    
#     print(der)

converged SCF energy = -195.830545310135


KeyboardInterrupt: 

## feature computation

In [10]:
rascal_hypers = {
    "interaction_cutoff": 4.5,
    "cutoff_smooth_width": 0.3,
    "max_radial": 4,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
#     "expansion_by_species_method": "user defined",
#     "global_species": [1,6,8,7]
    
}

cg = ClebschGordanReal(5)

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

### $\rho_{i}^{\otimes 3}$

In [23]:
rho1i = acdc_standardize_keys(rhoi)
rho1i = rho1i.keys_to_properties(['species_neighbor'])

rho2i = cg_increment(rho1i, rho1i, lcut=4, other_keys_match=["species_center"], clebsch_gordan=cg)
rho3i =  cg_increment(rho2i, rho1i, lcut=0, other_keys_match=["species_center"], clebsch_gordan=cg)

### $\rho_{i i_1 i_2}^{\otimes 1}$

In [26]:
gij =  acdc_standardize_keys(gij)
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg,lcut=4, other_keys_match=['species_center'])
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, lcut=0, other_keys_match = ['species_center'])


### check degeneracy of $\rho_{i}^{\otimes 3}$

In [30]:
np.linalg.norm(rho3i.block(species_center = 3, inversion_sigma = 1).values[0] - rho3i.block(species_center = 3, inversion_sigma = 1).values[1])

1.680772513045776e-22

In [29]:
# rhoii1i2_nu1.keys_to_properties('species_neighbor_1')
# rhoii1i2_nu1.keys_to_properties('species_neighbor_2')

In [49]:
rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).samples[36:72]#values.shape


Labels([(1, 0, 1, 1), (1, 0, 1, 2), (1, 0, 1, 3), (1, 0, 1, 4),
        (1, 0, 1, 5), (1, 0, 1, 6), (1, 0, 2, 1), (1, 0, 2, 2),
        (1, 0, 2, 3), (1, 0, 2, 4), (1, 0, 2, 5), (1, 0, 2, 6),
        (1, 0, 3, 1), (1, 0, 3, 2), (1, 0, 3, 3), (1, 0, 3, 4),
        (1, 0, 3, 5), (1, 0, 3, 6), (1, 0, 4, 1), (1, 0, 4, 2),
        (1, 0, 4, 3), (1, 0, 4, 4), (1, 0, 4, 5), (1, 0, 4, 6),
        (1, 0, 5, 1), (1, 0, 5, 2), (1, 0, 5, 3), (1, 0, 5, 4),
        (1, 0, 5, 5), (1, 0, 5, 6), (1, 0, 6, 1), (1, 0, 6, 2),
        (1, 0, 6, 3), (1, 0, 6, 4), (1, 0, 6, 5), (1, 0, 6, 6)],
       dtype=[('structure', '<i4'), ('center', '<i4'), ('neighbor_1', '<i4'), ('neighbor_2', '<i4')])

In [46]:
np.linalg.norm(rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=1, species_neighbor_b=12, inversion_sigma=1 ).values[:36])

2.5486800904721503e-07

In [48]:
# diff = np.sum(rhoii1i2_nu1.block(species_center = 6).values[:64], axis=0) - np.sum(rhoii1i2_nu1.block(species_center = 6).values[64:], axis=0)
diff = rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).values[:36] - rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).values[36:72]

np.linalg.norm(diff)

8.407296057023224e-08

## set up an encoder-decoder like architecture

In [319]:
import equistore
import torch
import equistore.operations as operations
from tqdm import tqdm
torch.set_default_dtype(torch.float64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  


class featDataset(torch.utils.data.Dataset):
    #Dataset class
    def __init__(self, feature_path, frames):
        #
        self.features = np.load(feature_path, mmap_mode = 'r')
        self.keys_features = equistore.io._labels_from_npz(self.features["keys"])
        self.currentkey = 0
        self.frames = frames

    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, structure_idx):
        feature_block, feature_key = self.generate_feature_block(self.features, structure_idx)        
        #samples_filter, target_block_samples = self.get_index_from_idx(self.target.block(self.currentkey).samples, structure_idx)
        structure = [frames[i] for i in structure_idx]
        #Modify feature_block to tensormap
        feature_map = TensorMap(feature_key, feature_block)
        return feature_map, structure

    
    def generate_feature_block(self, memmap, structure_idx):
        #Generate the block from npz file
        output = []
        if self.currentkey is None:
            feature_key = self.keys_features     
        else:
            feature_key = Labels(["order_nu", "inversion_sigma", "spherical_harmonics_l", "species_center", "species_neighbor_a", "species_neighbor_b"], 
                                 np.asarray(self.keys_features[self.currentkey].astype(list), dtype=np.int32).reshape(-1,6))

#             feature_key = [self.keys_features[self.currentkey]]
        for key in feature_key:
            block_index = list(self.keys_features).index(key)
            prefix = f"blocks/{block_index}/values"        
            block_samples = equistore.io._labels_from_npz(memmap[f"{prefix}/samples"])
            block_components = []
            for i in range(1):
                block_components.append(equistore.io._labels_from_npz(memmap[f"{prefix}/components/{i}"]))
            block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
             

            samples_filter, block_samples = self.get_index_from_idx(block_samples, structure_idx)

            block_data = memmap[f"{prefix}/data"][samples_filter]
            block = TensorBlock(block_data, block_samples, block_components, block_properties)
            output.append(block)
        return output, feature_key
    
    def get_n_properties(self, memmap, key):
        block_index = list(self.keys_features).index(key)
        prefix = f"blocks/{block_index}/values"  
        block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
        
        return len(block_properties)
    
    def get_index_from_idx(self, block_samples, structure_idx):
        #Get samples label from IDX
        samples = Labels(names = ['structure'], values = np.array(structure_idx).reshape(-1,1))
        all_samples = block_samples[['structure']].tolist()
        set_samples_to_slice = set(samples.tolist())
        samples_filter = np.array(
            [sample in set_samples_to_slice for sample in all_samples]
        )
        new_samples = block_samples[samples_filter]
        
        return samples_filter, new_samples
    
    def collate_output_values(blocks):
        feature_out = []
        for sample_output in blocks:
            feature_block, structure = sample_output
            for z in feature_block:
                feature_out.append(torch.tensor(z.values))

        return feature_out


In [315]:
def mse_loss(pred, target):
    return torch.mean((pred - target)**2)


class BlockEncoderDecoder(torch.nn.Module):
    def __init__(self, seed=123, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["input_size"], out_features=kwargs["hidden_size"]
        )
        self.encoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["intermediate_size"]
        )
        self.decoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["intermediate_size"], out_features=kwargs["hidden_size"]
        )
        self.decoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["input_size"]
        )
        self.to(device)
        
    def forward(self, x):
        print("from block forward", x.keys)
        x = torch.from_numpy(x.block().values)
        activation = self.encoder_hidden_layer(x)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

    
    def fit(self,traindata_loader, loss_function, optimizer_type, lr, reg, n_epochs):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
            threshold = 200
            scheduler_threshold = 50
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
            optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 10
            tol = 0
            history_step = 10
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            
            for x_data, structure in traindata_loader: 
                optimizer.zero_grad()
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        optimizer.zero_grad()
                        _pred = self.forward(x_data)                                        
                        _pred_loss = loss_function(_pred, x_data.block().values)
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                 
                        _reg_loss = self.get_regression_values(reg.item()) #Only works for 1 layer
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    optimizer.step(closure)

                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
#                     print(pred, x_data.block().values )
                    pred_loss = loss_function(pred, torch.from_numpy(x_data.block().values))
                    
                    #reg_loss = self.get_regression_values(reg.item())#Only works for 1 layer
                    new_loss = pred_loss #+ reg_loss
                    new_loss.backward()

                    optimizer.step()
                
            with torch.no_grad():
                current_loss = 0 
                for x_data, structure in traindata_loader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, torch.from_numpy(x_data.block().values)) #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg.item()) 
                new_loss = pred_loss + reg_loss
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0    
                    
                else:
                    trigger += 1
                    scheduler.step()
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output


    

class ThreeCenterModel(torch.nn.ModuleDict):
    
    def __init__(self, feature, hidden_size=128, intermediate_size=18):
        super().__init__()
        self.feature = feature
        self.loss_history={}
        self.models = torch.nn.ModuleDict()
        for k, b in self.feature:
            input_size = b.values.shape[-1]
#             print(k,input_size, hidden_size, intermediate_size)
            self.models[str(k)] = BlockEncoderDecoder(input_size=input_size, hidden_size=hidden_size, intermediate_size=intermediate_size)            
        self.to(device)
    
    def forward(self, feature):
        pred_blocks =[]
        for k, block in feature:
#             print(self[str(k)])
            
            recons = self[str(k)].forward(block) # expects a tensorMap
            
            newblock = TensorBlock(
                        values=recons.reshape(block.values.shape),
                        samples=block.samples,
                        components=block.components,
                        properties= block.properties
                    )
            pred_blocks.append(newblock) 
        
        keys = feature.keys
        pred_target = TensorMap(keys, pred_blocks)
        return(pred_target)
    
    def fit(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #Iterates through the keys of self.model, then for each key we will fit self.model[key] with data[key]
        total = len(self.models)
        for index, key in enumerate(self.feature.keys):
            print ("Now training on Block {} of {}".format(index, total))
            train_dataloader.dataset.currentkey = index
            
            loss_history_key = self.models[str(key)].fit(train_dataloader, loss_function, optimizer_type, lr, regularization_dict[str(key)], n_epochs)

            self.loss_history[str(key)] = loss_history_key

    
    


In [316]:
regularization_dict={}
for key in rhoii1i2_nu1.keys:
    regularization_dict[str(key)] = torch.tensor(0)

In [320]:
def collate_blocks(block_tuple):
    feature_tensor_map, structure_array = block_tuple[0]
    return feature_tensor_map, structure_array

train_feature_path ="./rhoii1i2n1_400.npz"
train = featDataset(train_feature_path, frames[:400])
train_Sampler = torch.utils.data.sampler.SequentialSampler(train)
train_BSampler = torch.utils.data.sampler.BatchSampler(train_Sampler, batch_size = 500, drop_last = False)

test_feature_path ="./rhoii1i2n1_400-500.npz"
test = featDataset(train_feature_path, frames[400:500])
test_Sampler = torch.utils.data.sampler.SequentialSampler(test)
test_BSampler = torch.utils.data.sampler.BatchSampler(test_Sampler, batch_size = 500, drop_last = False)

train_loader = torch.utils.data.DataLoader(train, sampler = train_BSampler, collate_fn = collate_blocks)
test_loader = torch.utils.data.DataLoader(test, sampler = test_BSampler, collate_fn = collate_blocks)

In [317]:
# rhoii1i2_nu1.keys[
rhoii1i2_nu1.blocks_matching(species_center=3)
# ]

[9, 10, 11, 12, 13, 14, 15, 16, 17, 36, 37, 38, 39, 40, 41, 42, 43, 44]

In [334]:
# ntrain = 400
# train_tmap = operations.slice(rhoii1i2_nu1, 
#                 samples = Labels(
#             names=["structure"],
#             values=np.array(
#                 list(range(ntrain))
#             ).reshape(-1,1), )
#                 )

# test_tmap = operations.slice(rhoii1i2_nu1, 
#                 samples = Labels(
#             names=["structure"],
#             values=np.array(
#                 list(range(ntrain, 500))
#             ).reshape(-1,1), )
#                 )

# from equistore.io import save, load
# # save('./rho3i_400.npz', rh)
# save('./rhoii1i2n1_400.npz', train_tmap)
# save('./rhoii1i2n1_400-500.npz', test_tmap)

/Users/jigyasa/miniconda3/lib/python3.8/site-packages/equistore/operations/slice.py:113: UserWarning: Some TensorBlocks in the sliced TensorMap are now empty, based on your choice of samples and/or properties to slice by. The keys of the empty TensorBlocks are:
 [(3,  1, 0,  1,  1, 12) (3,  1, 0,  1,  3, 12) (3,  1, 0,  1, 12,  1)
 (3,  1, 0,  1, 12,  3) (3,  1, 0,  1, 12, 12) (3,  1, 0,  3,  1, 12)
 (3,  1, 0,  3,  3, 12) (3,  1, 0,  3, 12,  1) (3,  1, 0,  3, 12,  3)
 (3,  1, 0,  3, 12, 12) (3,  1, 0, 12,  1,  1) (3,  1, 0, 12,  1,  3)
 (3,  1, 0, 12,  1, 12) (3,  1, 0, 12,  3,  1) (3,  1, 0, 12,  3,  3)
 (3,  1, 0, 12,  3, 12) (3,  1, 0, 12, 12,  1) (3,  1, 0, 12, 12,  3)
 (3, -1, 0,  1,  1, 12) (3, -1, 0,  1,  3, 12) (3, -1, 0,  1, 12,  1)
 (3, -1, 0,  1, 12,  3) (3, -1, 0,  1, 12, 12) (3, -1, 0,  3,  1, 12)
 (3, -1, 0,  3,  3, 12) (3, -1, 0,  3, 12,  1) (3, -1, 0,  3, 12,  3)
 (3, -1, 0,  3, 12, 12) (3, -1, 0, 12,  1,  1) (3, -1, 0, 12,  1,  3)
 (3, -1, 0, 12,  1, 12) (3, -1, 0, 12

In [322]:
model = ThreeCenterModel(rhoii1i2_nu1)

In [323]:
model.fit(train_loader, regularization_dict, optimizer_type="Adam", n_epochs=20, loss_function=mse_loss, lr=0.01)

Now training on Block 0 of 54


Epoch: 1:   5%|███▋                                                                      | 1/20 [00:00<00:02,  6.64it/s, lowest_loss=0.000333, pred_loss=0.000333, trigger=0]

from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]


Epoch: 2:  10%|███████▊                                                                      | 2/20 [00:00<00:02,  7.59it/s, lowest_loss=1.8e-5, pred_loss=1.8e-5, trigger=0]

from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]


...

from block forward [(3, 1, 0, 12, 1, 1)]
from block forward [(3, 1, 0, 12, 1, 1)]


Epoch: 7:  35%|██████████████████████████▌                                                 | 7/20 [00:02<00:04,  2.87it/s, lowest_loss=1.92e-8, pred_loss=2.12e-8, trigger=1]

from block forward [(3, 1, 0, 12, 1, 1)]
from block forward [(3, 1, 0, 12, 1, 1)]


Epoch: 7:  35%|██████████████████████████▌                                                 | 7/20 [00:02<00:05,  2.54it/s, lowest_loss=1.92e-8, pred_loss=2.12e-8, trigger=1]


KeyboardInterrupt: 

In [336]:
for x_data, structures in test_loader:
#     print(x_data)
    pred = model(x_data)
#     print ("Test error for mse_full is {}".format(mse_full(pred, )))

SyntaxError: unexpected EOF while parsing (1137940781.py, line 4)

In [ ]:
# aa=[]
# bb=[]
# for x, y in train_loader:
#     aa.append(x)
#     bb.append(y)
# from itertools import chain
# list(chain.from_iterable(bb))==frames[:400]